In [1]:
# Download the traffic speed data (HDF5 file)
!wget -N https://zenodo.org/records/4263971/files/pems-bay.h5?download=1 -O pems-bay.h5

# Download the sensor adjacency matrix (Pickle file)
!wget -N https://zenodo.org/records/4263971/files/adj_mx_bay.pkl?download=1 -O adj_mx_bay.pkl

print("--- Download Complete ---")
!ls -lh

for details.

--2025-11-02 12:35:50--  https://zenodo.org/records/4263971/files/pems-bay.h5?download=1
Resolving zenodo.org (zenodo.org)... 137.138.52.235, 188.185.48.75, 188.185.43.153, ...
Connecting to zenodo.org (zenodo.org)|137.138.52.235|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 135930936 (130M) [application/octet-stream]
Saving to: ‘pems-bay.h5’

pems-bay.h5         100%[===================>] 129.63M   463KB/s    in 5m 14s  

2025-11-02 12:41:05 (423 KB/s) - ‘pems-bay.h5’ saved [135930936/135930936]

for details.

--2025-11-02 12:41:05--  https://zenodo.org/records/4263971/files/adj_mx_bay.pkl?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.153, 188.185.48.75, 137.138.52.235, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1681480 (1.6M) [application/octet-stream]
Saving to: ‘adj_mx_bay.pkl’

adj_mx_bay.pkl      100%[===================>]   1.60M   422KB/

The error `NameError: name 'data_df' is not defined` indicates that the variable `data_df` was used before it was created. This variable is defined in the cell that loads the data from the HDF5 file. To fix this, you need to run the data loading cell first.

In [7]:
ls


adj_mx_bay.pkl  drive/  pems-bay.h5  sample_data/  scaler.pkl


In [8]:
from google.colab import drive
import os
import shutil

# 1️⃣ Mount Google Drive
drive.mount('/content/drive')

# 2️⃣ Create target folder in Drive
target_dir = "/content/drive/MyDrive/PEMS_BAY"
os.makedirs(target_dir, exist_ok=True)

# 3️⃣ Move the downloaded files into the new folder
files = ["pems-bay.h5", "adj_mx_bay.pkl"]

for file in files:
    if os.path.exists(f"/content/{file}"):
        shutil.move(f"/content/{file}", f"{target_dir}/{file}")
        print(f"✅ Moved {file} → {target_dir}")
    else:
        print(f"❌ File not found: {file}")

print("\n📂 Files now in Drive:")
!ls -lh /content/drive/MyDrive/PEMS_BAY


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Moved pems-bay.h5 → /content/drive/MyDrive/PEMS_BAY
✅ Moved adj_mx_bay.pkl → /content/drive/MyDrive/PEMS_BAY

📂 Files now in Drive:
total 132M
-rw------- 1 root root 1.7M Nov  2 12:41 adj_mx_bay.pkl
-rw------- 1 root root 130M Nov  2 12:41 pems-bay.h5


In [11]:
# loading dataset from GDrive

from google.colab import drive
import pandas as pd
import numpy as np
import pickle
import os
import h5py

# --- Mount Google Drive ---
drive.mount('/content/drive')

DATA_DIR = "/content/drive/MyDrive/PEMS_BAY"

traffic_file = os.path.join(DATA_DIR, "pems-bay.h5")
adj_file = os.path.join(DATA_DIR, "adj_mx_bay.pkl")

print(f"Using dataset from: {DATA_DIR}\n")

# --- Load the Traffic Data ---
try:
    data_df = pd.read_hdf(traffic_file)
except Exception as e:
    print(f"Error reading with pandas, trying h5py: {e}")
    with h5py.File(traffic_file, 'r') as f:
        print("HDF5 Keys found:", list(f.keys()))
        # PEMS-BAY files usually contain 'speed' key
        key = list(f.keys())[0]  # taking first available key
        print(f"Using key: {key}")
        data = f[key][:]
        data_df = pd.DataFrame(data)

print("--- Traffic Data (Speeds) ---")
print(f"Data shape: {data_df.shape}")
print("(Timesteps, Sensors)")
print("\nFirst 5 rows and 5 columns:")
print(data_df.iloc[:5, :5])
print("\n")

# --- Load the Adjacency Matrix ---
with open(adj_file, 'rb') as f:
    adj_data = pickle.load(f, encoding='latin1')

sensor_ids = adj_data[0]
sensor_id_to_ind = adj_data[1]
adj_matrix = adj_data[2]

print("--- Adjacency Matrix ---")
print(f"Matrix shape: {adj_matrix.shape}")
print(f"Num sensors: {len(sensor_ids)}\n")
print(f"This is a {adj_matrix.shape[0]}x{adj_matrix.shape[1]} adjacency matrix representing sensor connectivity.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using dataset from: /content/drive/MyDrive/PEMS_BAY

--- Traffic Data (Speeds) ---
Data shape: (52116, 325)
(Timesteps, Sensors)

First 5 rows and 5 columns:
sensor_id            400001  400017  400030  400040  400045
2017-01-01 00:00:00    71.4    67.8    70.5    67.4    68.8
2017-01-01 00:05:00    71.6    67.5    70.6    67.5    68.7
2017-01-01 00:10:00    71.6    67.6    70.2    67.4    68.7
2017-01-01 00:15:00    71.1    67.5    70.3    68.0    68.5
2017-01-01 00:20:00    71.7    67.8    70.2    68.1    68.4


--- Adjacency Matrix ---
Matrix shape: (325, 325)
Num sensors: 325

This is a 325x325 adjacency matrix representing sensor connectivity.


In [13]:

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pickle

# --- 1. Load Data (assuming data_df from previous step) ---
# If you lost the variable, uncomment the line below:
# data_df = pd.read_hdf('pems-bay.h5')

# Convert to NumPy array for efficiency
data_array = data_df.values.astype(np.float32)

# --- 2. Define Parameters ---
# We'll use 12 time steps (1 hour) to predict the next 12 (1 hour)
SEQ_LEN = 12  # Input sequence length (lookback)
HORIZON = 12  # Output sequence length (prediction)
N_SENSORS = data_array.shape[1]  # Should be 325

# --- 3. Normalize Data ---
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data_array)

print(f"Original data shape: {data_array.shape}")
print(f"Normalized data shape: {data_normalized.shape}")
print(f"Data min/max before: {data_array.min()}, {data_array.max()}")
print(f"Data min/max after: {data_normalized.min()}, {data_normalized.max()}")

# --- 4. Save the Scaler ---
# This is VITAL for your API. We need it to
# transform new input data and to 'invert' the model's
# output (which will be in the 0-1 range) back to real speed values.
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("\nScaler saved to 'scaler.pkl'")

Original data shape: (52116, 325)
Normalized data shape: (52116, 325)
Data min/max before: 0.0, 85.0999984741211
Data min/max after: 0.0, 1.0000001192092896

Scaler saved to 'scaler.pkl'


In [14]:
# creating sliding windows
def create_sliding_windows(data, seq_len, horizon):
    """
    Converts a time series array into X (input) and y (target) windows.
    """
    X = []
    y = []

    # Total length of one sample (input + output)
    total_len = seq_len + horizon

    # Iterate from the first possible sample start up to the last
    # (data.shape[0] - total_len + 1) is the number of samples we can create
    for i in range(data.shape[0] - total_len + 1):
        # Input window: from i to i + seq_len
        input_window = data[i : i + seq_len]
        X.append(input_window)

        # Target window: from i + seq_len to i + seq_len + horizon
        target_window = data[i + seq_len : i + seq_len + horizon]
        y.append(target_window)

    # Convert lists to NumPy arrays
    # Shape of X: (num_samples, seq_len, num_features)
    # Shape of y: (num_samples, horizon, num_features)
    return np.array(X), np.array(y)

# Create the windows
X_data, y_data = create_sliding_windows(data_normalized, SEQ_LEN, HORIZON)

print("\n--- Sliding Windows Created ---")
print(f"Total normalized data shape: {data_normalized.shape}")
print(f"Resulting X shape: {X_data.shape}")
print(f"Resulting y shape: {y_data.shape}")


--- Sliding Windows Created ---
Total normalized data shape: (52116, 325)
Resulting X shape: (52093, 12, 325)
Resulting y shape: (52093, 12, 325)


In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pickle

# --- 1. Load Data (assuming data_df from previous step) ---
# If you lost the variable, uncomment the line below:
# data_df = pd.read_hdf('pems-bay.h5')

# Convert to NumPy array for efficiency
data_array = data_df.values.astype(np.float32)

# --- 2. Define Parameters ---
# We'll use 12 time steps (1 hour) to predict the next 12 (1 hour)
SEQ_LEN = 12  # Input sequence length (lookback)
HORIZON = 12  # Output sequence length (prediction)
N_SENSORS = data_array.shape[1]  # Should be 325

# --- 3. Normalize Data ---
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data_array)

print(f"Original data shape: {data_array.shape}")
print(f"Normalized data shape: {data_normalized.shape}")
print(f"Data min/max before: {data_array.min()}, {data_array.max()}")
print(f"Data min/max after: {data_normalized.min()}, {data_normalized.max()}")

# --- 4. Save the Scaler ---
# This is VITAL for your API. We need it to
# transform new input data and to 'invert' the model's
# output (which will be in the 0-1 range) back to real speed values.
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("\nScaler saved to 'scaler.pkl'")

Original data shape: (52116, 325)
Normalized data shape: (52116, 325)
Data min/max before: 0.0, 85.0999984741211
Data min/max after: 0.0, 1.0000001192092896

Scaler saved to 'scaler.pkl'


In [15]:
# We split chronologically, not randomly.
# 70% Train, 10% Validation, 20% Test

total_samples = X_data.shape[0]

# 70% for training
train_split = int(total_samples * 0.7)
# 80% (70% + 10%) for train + val
val_split = int(total_samples * 0.8)

# Training set
X_train = X_data[:train_split]
y_train = y_data[:train_split]

# Validation set
X_val = X_data[train_split:val_split]
y_val = y_data[train_split:val_split]

# Test set
X_test = X_data[val_split:]
y_test = y_data[val_split:]

print("\n--- Data Split ---")
print(f"X_train shape: {X_train.shape} | y_train shape: {y_train.shape}")
print(f"X_val shape:   {X_val.shape} | y_val shape:   {y_val.shape}")
print(f"X_test shape:  {X_test.shape} | y_test shape:  {y_test.shape}")


--- Data Split ---
X_train shape: (36465, 12, 325) | y_train shape: (36465, 12, 325)
X_val shape:   (5209, 12, 325) | y_val shape:   (5209, 12, 325)
X_test shape:  (10419, 12, 325) | y_test shape:  (10419, 12, 325)


In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, TimeDistributed, Reshape

# Define the CNN model
model = Sequential([
    # Reshape the input to be treated as an image (time steps, sensors)
    Reshape((SEQ_LEN, N_SENSORS, 1), input_shape=(SEQ_LEN, N_SENSORS)),

    # Convolutional layers
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),

    # Flatten the output of the convolutional layers
    Flatten(),

    # Dense layers
    Dense(128, activation='relu'),
    Dense(SEQ_LEN * N_SENSORS) # Output layer with size matching the target shape (horizon * sensors)
])

# Reshape the output to match the target shape (horizon, sensors)
model.add(Reshape((HORIZON, N_SENSORS)))


# Compile the model
model.compile(optimizer='adam', loss='mse')

# Display the model summary
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_2 (Reshape)             │ (None, 12, 325, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 325, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 162, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 6, 162, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 3, 81, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 15552)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     1,990,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3900)           │       503,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_3 (Reshape)             │ (None, 12, 325)        │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,512,700 (9.59 MB)

 Trainable params: 2,512,700 (9.59 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=5, # You can adjust the number of epochs
    batch_size=32, # You can adjust the batch size
    validation_data=(X_val, y_val)
)

Epoch 1/5
1140/1140 ━━━━━━━━━━━━━━━━━━━━ 241s 210ms/step - loss: 0.0289 - val_loss: 0.0057
Epoch 2/5
1140/1140 ━━━━━━━━━━━━━━━━━━━━ 260s 208ms/step - loss: 0.0044 - val_loss: 0.0052
Epoch 3/5
1140/1140 ━━━━━━━━━━━━━━━━━━━━ 259s 206ms/step - loss: 0.0038 - val_loss: 0.0046
Epoch 4/5
1140/1140 ━━━━━━━━━━━━━━━━━━━━ 261s 205ms/step - loss: 0.0034 - val_loss: 0.0044
Epoch 5/5
1140/1140 ━━━━━━━━━━━━━━━━━━━━ 258s 202ms/step - loss: 0.0032 - val_loss: 0.0042


The model has been trained. You can now evaluate it on the test set or use it to make predictions.

Shape meaning (example):

X_data.shape is (52093, 12, 325): We have 52,093 training samples. Each sample is 12 time steps long (input hour) and includes all 325 sensors.

y_data.shape is (52093, 12, 325): For each sample, we have a corresponding target. The target is the next 12 time steps (future hour) for all 325 sensors.

In [25]:
# ============================================================================
# FIX OUT-OF-RANGE DATA - ADD THIS CELL TO YOUR NOTEBOOK
# ============================================================================
# Run this cell BEFORE calling run_complete_testing_suite()
# This will automatically fix any out-of-range values in your data

import numpy as np

print("\n" + "="*80)
print("🔧 FIXING OUT-OF-RANGE DATA")
print("="*80)

# Store original data info
print("\n📊 Before Fix:")
print(f"  X_train range: [{X_train.min():.10f}, {X_train.max():.10f}]")
print(f"  X_val range:   [{X_val.min():.10f}, {X_val.max():.10f}]")
print(f"  X_test range:  [{X_test.min():.10f}, {X_test.max():.10f}]")
print(f"  y_train range: [{y_train.min():.10f}, {y_train.max():.10f}]")
print(f"  y_val range:   [{y_val.min():.10f}, {y_val.max():.10f}]")
print(f"  y_test range:  [{y_test.min():.10f}, {y_test.max():.10f}]")

# Count out-of-range values
out_of_range_before = (
    ((X_train < 0) | (X_train > 1)).sum() +
    ((X_val < 0) | (X_val > 1)).sum() +
    ((X_test < 0) | (X_test > 1)).sum() +
    ((y_train < 0) | (y_train > 1)).sum() +
    ((y_val < 0) | (y_val > 1)).sum() +
    ((y_test < 0) | (y_test > 1)).sum()
)

print(f"\n⚠️  Out-of-range values found: {out_of_range_before}")

# Step 1: Fix NaN values
print("\n✓ Fixing NaN values...")
nan_count = (
    np.isnan(X_train).sum() + np.isnan(X_val).sum() + np.isnan(X_test).sum() +
    np.isnan(y_train).sum() + np.isnan(y_val).sum() + np.isnan(y_test).sum()
)

if nan_count > 0:
    print(f"  Found {nan_count} NaN values - replacing with 0.5 (normalized mean)...")

    X_train = np.nan_to_num(X_train, nan=0.5)
    X_val = np.nan_to_num(X_val, nan=0.5)
    X_test = np.nan_to_num(X_test, nan=0.5)
    y_train = np.nan_to_num(y_train, nan=0.5)
    y_val = np.nan_to_num(y_val, nan=0.5)
    y_test = np.nan_to_num(y_test, nan=0.5)
    print(f"  ✅ NaN values fixed")
else:
    print(f"  ✅ No NaN values found")

# Step 2: Fix Infinite values
print("\n✓ Fixing infinite values...")
inf_count = (
    np.isinf(X_train).sum() + np.isinf(X_val).sum() + np.isinf(X_test).sum() +
    np.isinf(y_train).sum() + np.isinf(y_val).sum() + np.isinf(y_test).sum()
)

if inf_count > 0:
    print(f"  Found {inf_count} infinite values...")

    # Replace positive inf with 1, negative inf with 0
    X_train = np.where(np.isposinf(X_train), 1.0, np.where(np.isneginf(X_train), 0.0, X_train))
    X_val = np.where(np.isposinf(X_val), 1.0, np.where(np.isneginf(X_val), 0.0, X_val))
    X_test = np.where(np.isposinf(X_test), 1.0, np.where(np.isneginf(X_test), 0.0, X_test))
    y_train = np.where(np.isposinf(y_train), 1.0, np.where(np.isneginf(y_train), 0.0, y_train))
    y_val = np.where(np.isposinf(y_val), 1.0, np.where(np.isneginf(y_val), 0.0, y_val))
    y_test = np.where(np.isposinf(y_test), 1.0, np.where(np.isneginf(y_test), 0.0, y_test))
    print(f"  ✅ Infinite values fixed")
else:
    print(f"  ✅ No infinite values found")

# Step 3: Clip values to [0, 1]
print("\n✓ Clipping values to [0, 1] range...")

X_train = np.clip(X_train, 0, 1)
X_val = np.clip(X_val, 0, 1)
X_test = np.clip(X_test, 0, 1)
y_train = np.clip(y_train, 0, 1)
y_val = np.clip(y_val, 0, 1)
y_test = np.clip(y_test, 0, 1)

print(f"  ✅ Clipping complete")

# Verify the fix
print("\n📊 After Fix:")
print(f"  X_train range: [{X_train.min():.10f}, {X_train.max():.10f}]")
print(f"  X_val range:   [{X_val.min():.10f}, {X_val.max():.10f}]")
print(f"  X_test range:  [{X_test.min():.10f}, {X_test.max():.10f}]")
print(f"  y_train range: [{y_train.min():.10f}, {y_train.max():.10f}]")
print(f"  y_val range:   [{y_val.min():.10f}, {y_val.max():.10f}]")
print(f"  y_test range:  [{y_test.min():.10f}, {y_test.max():.10f}]")

# Final verification
all_valid = (
    (X_train.min() >= 0 and X_train.max() <= 1) and
    (X_val.min() >= 0 and X_val.max() <= 1) and
    (X_test.min() >= 0 and X_test.max() <= 1) and
    (y_train.min() >= 0 and y_train.max() <= 1) and
    (y_val.min() >= 0 and y_val.max() <= 1) and
    (y_test.min() >= 0 and y_test.max() <= 1) and
    not np.isnan(X_train).any() and not np.isnan(X_val).any() and not np.isnan(X_test).any() and
    not np.isnan(y_train).any() and not np.isnan(y_val).any() and not np.isnan(y_test).any()
)

if all_valid:
    print("\n" + "="*80)
    print("✅ ALL DATA FIXED AND VALIDATED!")
    print("   You can now safely run: run_complete_testing_suite(...)")
    print("="*80)
else:
    print("\n" + "="*80)
    print("⚠️  WARNING: Some data quality issues remain")
    print("="*80)



🔧 FIXING OUT-OF-RANGE DATA

📊 Before Fix:
  X_train range: [0.0000000000, 1.0000001192]
  X_val range:   [0.0000000000, 1.0000000000]
  X_test range:  [0.0000000000, 1.0000001192]
  y_train range: [0.0000000000, 1.0000001192]
  y_val range:   [0.0000000000, 1.0000000000]
  y_test range:  [0.0000000000, 1.0000001192]

⚠️  Out-of-range values found: 96

✓ Fixing NaN values...
  ✅ No NaN values found

✓ Fixing infinite values...
  ✅ No infinite values found

✓ Clipping values to [0, 1] range...
  ✅ Clipping complete

📊 After Fix:
  X_train range: [0.0000000000, 1.0000000000]
  X_val range:   [0.0000000000, 1.0000000000]
  X_test range:  [0.0000000000, 1.0000000000]
  y_train range: [0.0000000000, 1.0000000000]
  y_val range:   [0.0000000000, 1.0000000000]
  y_test range:  [0.0000000000, 1.0000000000]

✅ ALL DATA FIXED AND VALIDATED!
   You can now safely run: run_complete_testing_suite(...)


In [29]:
# ============================================================================
# FIX: Proper Scaler Usage and Denormalization
# ============================================================================
# Add this BEFORE running the testing suite

print("\n" + "="*80)
print("FIXING SCALER FOR PROPER DENORMALIZATION")
print("="*80)

# Step 1: Understand your scaler
print("\n📋 Analyzing Scaler...")
print(f"  Scaler type: {type(scaler)}")
print(f"  Scaler data_min_ shape: {scaler.data_min_.shape}")
print(f"  Scaler data_max_ shape: {scaler.data_max_.shape}")
print(f"  Scaler feature_range: {scaler.feature_range}")

# Step 2: Check how scaler was fit
original_features = scaler.data_min_.shape[0]
print(f"  Number of features scaler was fit on: {original_features}")

# Step 3: Create a helper function for proper denormalization
def denormalize_data(normalized_data, scaler, original_shape=325):
    """
    Properly denormalize data that was normalized column-wise

    Args:
        normalized_data: The normalized data (can be any shape)
        scaler: The fitted MinMaxScaler
        original_shape: Number of features (sensors) in original data

    Returns:
        Denormalized data
    """
    original_shape_flat = normalized_data.shape

    # Reshape to (n_samples, n_features) for inverse_transform
    normalized_reshaped = normalized_data.reshape(-1, original_shape)

    # Inverse transform
    denormalized = scaler.inverse_transform(normalized_reshaped)

    # Reshape back to original shape
    denormalized = denormalized.reshape(original_shape_flat)

    return denormalized


# Test the denormalization function
print("\n✓ Testing denormalization function...")
test_sample = y_test[0:2].copy()  # Get 2 samples
print(f"  Original test shape: {test_sample.shape}")
print(f"  Value range before denorm: [{test_sample.min():.6f}, {test_sample.max():.6f}]")

denorm_test = denormalize_data(test_sample, scaler, original_shape=N_SENSORS)
print(f"  Denormalized shape: {denorm_test.shape}")
print(f"  Value range after denorm: [{denorm_test.min():.6f}, {denorm_test.max():.6f}]")
print(f"  ✅ Denormalization working correctly")

# Step 4: Update the testing module with this function
print("\n✓ Creating updated metrics calculation...")

def calculate_metrics_fixed(y_true, y_pred, scaler, n_sensors=325):
    """
    Calculate metrics with proper denormalization
    """
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

    y_true_flat = y_true.reshape(-1)
    y_pred_flat = y_pred.reshape(-1)

    metrics = {}

    # Normalized metrics
    metrics['mse_normalized'] = float(mean_squared_error(y_true_flat, y_pred_flat))
    metrics['rmse_normalized'] = float(np.sqrt(metrics['mse_normalized']))
    metrics['mae_normalized'] = float(mean_absolute_error(y_true_flat, y_pred_flat))
    metrics['r2_normalized'] = float(r2_score(y_true_flat, y_pred_flat))

    # Denormalized metrics
    if scaler is not None:
        try:
            # Reshape properly for scaler
            y_true_reshaped = y_true.reshape(-1, n_sensors)
            y_pred_reshaped = y_pred.reshape(-1, n_sensors)

            # Inverse transform
            y_true_denorm = scaler.inverse_transform(y_true_reshaped).reshape(-1)
            y_pred_denorm = scaler.inverse_transform(y_pred_reshaped).reshape(-1)

            metrics['mse_denorm'] = float(mean_squared_error(y_true_denorm, y_pred_denorm))
            metrics['rmse_denorm'] = float(np.sqrt(metrics['mse_denorm']))
            metrics['mae_denorm'] = float(mean_absolute_error(y_true_denorm, y_pred_denorm))
            metrics['r2_denorm'] = float(r2_score(y_true_denorm, y_pred_denorm))

            # MAPE
            mask = y_true_denorm != 0
            if mask.sum() > 0:
                mape = np.mean(np.abs((y_true_denorm[mask] - y_pred_denorm[mask]) /
                                      y_true_denorm[mask])) * 100
            else:
                mape = 0.0
            metrics['mape'] = float(mape)

            metrics['y_true_denorm_stats'] = {
                'min': float(y_true_denorm.min()),
                'max': float(y_true_denorm.max()),
                'mean': float(y_true_denorm.mean()),
                'std': float(y_true_denorm.std())
            }
            metrics['y_pred_denorm_stats'] = {
                'min': float(y_pred_denorm.min()),
                'max': float(y_pred_denorm.max()),
                'mean': float(y_pred_denorm.mean()),
                'std': float(y_pred_denorm.std())
            }

            print("  ✅ Denormalization successful")

        except Exception as e:
            print(f"  ⚠️  Denormalization failed: {str(e)[:100]}...")
            print(f"  Falling back to normalized metrics only")

    return metrics


# Test it on a small sample
print("\n✓ Testing fixed metrics calculation on sample...")
y_pred_sample = model.predict(X_test[0:10], verbose=0)
metrics_sample = calculate_metrics_fixed(y_test[0:10], y_pred_sample, scaler, N_SENSORS)

print(f"  RMSE (normalized): {metrics_sample['rmse_normalized']:.6f}")
if 'rmse_denorm' in metrics_sample:
    print(f"  RMSE (denorm - mph): {metrics_sample['rmse_denorm']:.4f}")
    print(f"  MAE (denorm - mph): {metrics_sample['mae_denorm']:.4f}")
    print(f"  MAPE: {metrics_sample['mape']:.2f}%")
    print(f"  R²: {metrics_sample['r2_denorm']:.6f}")
    print(f"  ✅ All metrics calculated successfully!")

print("\n" + "="*80)
print("✅ SCALER FIX COMPLETE")
print("="*80)

print("\n💡 NEXT STEPS:")
print("  1. Use calculate_metrics_fixed() instead of calculate_metrics()")
print("  2. Or, update cnn_model_testing.py with the proper denormalization")
print("  3. Run the testing suite again")



FIXING SCALER FOR PROPER DENORMALIZATION

📋 Analyzing Scaler...
  Scaler type: <class 'sklearn.preprocessing._data.MinMaxScaler'>
  Scaler data_min_ shape: (325,)
  Scaler data_max_ shape: (325,)
  Scaler feature_range: (0, 1)
  Number of features scaler was fit on: 325

✓ Testing denormalization function...
  Original test shape: (2, 12, 325)
  Value range before denorm: [0.114925, 0.994764]
  Denormalized shape: (2, 12, 325)
  Value range after denorm: [9.300000, 76.000000]
  ✅ Denormalization working correctly

✓ Creating updated metrics calculation...

✓ Testing fixed metrics calculation on sample...
  ✅ Denormalization successful
  RMSE (normalized): 0.079671
  RMSE (denorm - mph): 5.8962
  MAE (denorm - mph): 3.7445
  MAPE: 7.59%
  R²: 0.652511
  ✅ All metrics calculated successfully!

✅ SCALER FIX COMPLETE

💡 NEXT STEPS:
  1. Use calculate_metrics_fixed() instead of calculate_metrics()
  2. Or, update cnn_model_testing.py with the proper denormalization
  3. Run the testing sui

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model
from typing import Tuple, Dict
import warnings
warnings.filterwarnings('ignore')


# ============================================================================
# PHASE 1: DATA QUALITY TESTING
# ============================================================================

class DataQualityTests:
    """Test data integrity before training"""

    @staticmethod
    def test_data_completeness(X: np.ndarray, y: np.ndarray) -> Dict:
        """Check for missing values and data integrity"""
        results = {}

        # Check for NaNs
        X_has_nan = np.isnan(X).any()
        y_has_nan = np.isnan(y).any()

        results['X_has_nan'] = X_has_nan
        results['y_has_nan'] = y_has_nan
        results['X_nan_count'] = np.isnan(X).sum()
        results['y_nan_count'] = np.isnan(y).sum()

        # Check for infinities
        results['X_has_inf'] = np.isinf(X).any()
        results['y_has_inf'] = np.isinf(y).any()

        return results

    @staticmethod
    def test_data_ranges(X: np.ndarray, y: np.ndarray,
                        expected_min=0, expected_max=1, auto_fix=True) -> Dict:
        """Verify data is within expected ranges"""
        results = {}

        results['X_min'] = float(X.min())
        results['X_max'] = float(X.max())
        results['y_min'] = float(y.min())
        results['y_max'] = float(y.max())

        X_in_range = (X.min() >= expected_min and X.max() <= expected_max)
        y_in_range = (y.min() >= expected_min and y.max() <= expected_max)

        results['X_in_range'] = X_in_range
        results['y_in_range'] = y_in_range
        results['auto_fix_applied'] = False
        results['values_clipped'] = 0

        # Auto-fix: Clip values to [0, 1] if they're slightly out of range
        if auto_fix and (not X_in_range or not y_in_range):
            # Only auto-fix if values are slightly out of range (likely floating point errors)
            if (X.min() >= -0.1 and X.max() <= 1.1 and
                y.min() >= -0.1 and y.max() <= 1.1):
                values_clipped = ((X < 0) | (X > 1)).sum() + ((y < 0) | (y > 1)).sum()
                X[:] = np.clip(X, 0, 1)
                y[:] = np.clip(y, 0, 1)
                results['auto_fix_applied'] = True
                results['values_clipped'] = values_clipped
                results['X_in_range'] = True
                results['y_in_range'] = True

        return results

    @staticmethod
    def test_shapes(X_train, X_val, X_test, y_train, y_val, y_test,
                   expected_seq_len=12, expected_horizon=12,
                   expected_sensors=325) -> Dict:
        """Verify all shapes are correct"""
        results = {}

        # Expected shapes
        results['X_train_shape'] = X_train.shape
        results['X_val_shape'] = X_val.shape
        results['X_test_shape'] = X_test.shape
        results['y_train_shape'] = y_train.shape
        results['y_val_shape'] = y_val.shape
        results['y_test_shape'] = y_test.shape

        # Validation
        results['seq_len_correct'] = (X_train.shape[1] == expected_seq_len)
        results['horizon_correct'] = (y_train.shape[1] == expected_horizon)
        results['sensors_correct'] = (X_train.shape[2] == expected_sensors)
        results['batch_dims_match'] = (X_train.shape[0] == y_train.shape[0])

        return results

    @staticmethod
    def test_data_distribution(X: np.ndarray, y: np.ndarray) -> Dict:
        """Check data statistics and distribution"""
        results = {}

        results['X_mean'] = float(X.mean())
        results['X_std'] = float(X.std())
        results['y_mean'] = float(y.mean())
        results['y_std'] = float(y.std())

        # Check for extreme outliers (> 3 std deviations)
        X_outliers = np.abs(X - X.mean()) > 3 * X.std()
        y_outliers = np.abs(y - y.mean()) > 3 * y.std()

        results['X_outlier_percentage'] = float((X_outliers.sum() / X.size) * 100)
        results['y_outlier_percentage'] = float((y_outliers.sum() / y.size) * 100)

        return results

    @staticmethod
    def run_all_tests(X_train, X_val, X_test, y_train, y_val, y_test) -> None:
        """Run all data quality tests and print report"""
        print("\n" + "="*70)
        print("PHASE 1: DATA QUALITY TESTING")
        print("="*70)

        # Test completeness
        print("\n✓ Testing Data Completeness...")
        completeness = DataQualityTests.test_data_completeness(X_train, y_train)
        assert not completeness['X_has_nan'], "❌ NaNs found in X_train"
        assert not completeness['y_has_nan'], "❌ NaNs found in y_train"
        print("  ✅ No NaN values found")

        # Test ranges with auto-fix
        print("\n✓ Testing Data Ranges...")
        ranges = DataQualityTests.test_data_ranges(X_train, y_train, auto_fix=True)

        if ranges['auto_fix_applied']:
            print(f"  ⚠️  Auto-fixed out-of-range values: {ranges['values_clipped']} clipped")
            print(f"  X range (after fix): [{ranges['X_min']:.4f}, {ranges['X_max']:.4f}]")
            print(f"  y range (after fix): [{ranges['y_min']:.4f}, {ranges['y_max']:.4f}]")
            # Also fix validation and test sets
            DataQualityTests.test_data_ranges(X_val, y_val, auto_fix=True)
            DataQualityTests.test_data_ranges(X_test, y_test, auto_fix=True)
            print(f"  ✅ Applied same fix to validation and test sets")
        else:
            print(f"  X range: [{ranges['X_min']:.4f}, {ranges['X_max']:.4f}]")
            print(f"  y range: [{ranges['y_min']:.4f}, {ranges['y_max']:.4f}]")

        assert ranges['X_in_range'], "❌ X data out of expected range"
        assert ranges['y_in_range'], "❌ y data out of expected range"
        print("  ✅ Data in expected range")

        # Test shapes
        print("\n✓ Testing Data Shapes...")
        shapes = DataQualityTests.test_shapes(X_train, X_val, X_test,
                                             y_train, y_val, y_test)
        assert shapes['seq_len_correct'], "❌ Sequence length incorrect"
        assert shapes['horizon_correct'], "❌ Horizon incorrect"
        assert shapes['sensors_correct'], "❌ Sensor count incorrect"
        print(f"  X_train: {shapes['X_train_shape']}")
        print(f"  y_train: {shapes['y_train_shape']}")
        print(f"  X_val:   {shapes['X_val_shape']}")
        print(f"  X_test:  {shapes['X_test_shape']}")
        print("  ✅ All shapes correct")

        # Test distribution
        print("\n✓ Testing Data Distribution...")
        dist = DataQualityTests.test_data_distribution(X_train, y_train)
        print(f"  X mean: {dist['X_mean']:.4f} ± {dist['X_std']:.4f}")
        print(f"  y mean: {dist['y_mean']:.4f} ± {dist['y_std']:.4f}")
        print(f"  X outliers: {dist['X_outlier_percentage']:.2f}%")
        print(f"  y outliers: {dist['y_outlier_percentage']:.2f}%")
        if dist['X_outlier_percentage'] > 5:
            print("  ⚠️  High outlier percentage in X")
        else:
            print("  ✅ Outlier percentage reasonable")

        print("\n" + "="*70)
        print("✅ ALL DATA QUALITY TESTS PASSED")
        print("="*70)


# ============================================================================
# PHASE 3: PERFORMANCE METRICS EVALUATION
# ============================================================================

class PerformanceMetrics:
    """Calculate comprehensive performance metrics"""

    @staticmethod
    def calculate_metrics(y_true: np.ndarray, y_pred: np.ndarray,
                         scaler=None, n_sensors: int = 325) -> Dict:
        """Calculate all relevant metrics

        Args:
            y_true: True values (can be 2D or 3D)
            y_pred: Predicted values (can be 2D or 3D)
            scaler: MinMaxScaler object (fitted on original data)
            n_sensors: Number of sensors/features in original data
        """
        # Work with flattened arrays for easier computation
        y_true_flat = y_true.reshape(-1)
        y_pred_flat = y_pred.reshape(-1)

        metrics = {}

        # Normalized space metrics (0-1 range)
        metrics['mse_normalized'] = float(mean_squared_error(y_true_flat, y_pred_flat))
        metrics['rmse_normalized'] = float(np.sqrt(metrics['mse_normalized']))
        metrics['mae_normalized'] = float(mean_absolute_error(y_true_flat, y_pred_flat))
        metrics['r2_normalized'] = float(r2_score(y_true_flat, y_pred_flat))

        # Denormalized metrics (actual speed values)
        if scaler is not None:
            # The scaler was fit on (timesteps, sensors) shaped data
            # We need to reshape our flat data back to (n_samples, n_sensors) format
            try:
                # Reshape flat data to proper 2D format for inverse_transform
                # The scaler expects shape (n_samples, n_features)
                # Since data was flattened, we reshape to (-1, n_sensors)
                y_true_reshaped = y_true_flat.reshape(-1, n_sensors)
                y_pred_reshaped = y_pred_flat.reshape(-1, n_sensors)

                # Inverse transform (scaler was fit on full 2D data)
                y_true_denorm = scaler.inverse_transform(y_true_reshaped)
                y_pred_denorm = scaler.inverse_transform(y_pred_reshaped)

                # Flatten back for metric calculation
                y_true_denorm = y_true_denorm.flatten()
                y_pred_denorm = y_pred_denorm.flatten()

            except (ValueError, AttributeError) as e:
                # Fallback: manual denormalization using scaler's min/max
                print(f"  ⚠️  Using manual denormalization (reshape failed: {str(e)[:50]}...)")
                # Manual formula: y_denorm = y_norm * (max - min) + min
                # For multi-dimensional data, use the first feature's scale
                scale_factor = scaler.data_max_ - scaler.data_min_
                min_val = scaler.data_min_

                # Average scale across all sensors
                avg_scale = scale_factor.mean()
                avg_min = min_val.mean()

                y_true_denorm = y_true_flat * avg_scale + avg_min
                y_pred_denorm = y_pred_flat * avg_scale + avg_min

            metrics['mse_denorm'] = float(mean_squared_error(y_true_denorm, y_pred_denorm))
            metrics['rmse_denorm'] = float(np.sqrt(metrics['mse_denorm']))
            metrics['mae_denorm'] = float(mean_absolute_error(y_true_denorm, y_pred_denorm))
            metrics['r2_denorm'] = float(r2_score(y_true_denorm, y_pred_denorm))

            # MAPE (Mean Absolute Percentage Error)
            mask = y_true_denorm != 0
            if mask.sum() > 0:
                mape = np.mean(np.abs((y_true_denorm[mask] - y_pred_denorm[mask]) /
                                      y_true_denorm[mask])) * 100
            else:
                mape = 0.0
            metrics['mape'] = float(mape)

            metrics['y_true_denorm_stats'] = {
                'min': float(y_true_denorm.min()),
                'max': float(y_true_denorm.max()),
                'mean': float(y_true_denorm.mean()),
                'std': float(y_true_denorm.std())
            }
            metrics['y_pred_denorm_stats'] = {
                'min': float(y_pred_denorm.min()),
                'max': float(y_pred_denorm.max()),
                'mean': float(y_pred_denorm.mean()),
                'std': float(y_pred_denorm.std())
            }

        return metrics

    @staticmethod
    def print_metrics_report(metrics: Dict, dataset_name: str = "Dataset") -> None:
        """Print formatted metrics report"""
        print(f"\n{'='*70}")
        print(f"PERFORMANCE METRICS - {dataset_name}")
        print(f"{'='*70}")

        print(f"\n📊 Normalized Metrics (0-1 range):")
        print(f"  MSE:  {metrics['mse_normalized']:.6f}")
        print(f"  RMSE: {metrics['rmse_normalized']:.6f}")
        print(f"  MAE:  {metrics['mae_normalized']:.6f}")
        print(f"  R²:   {metrics['r2_normalized']:.6f}")

        if 'rmse_denorm' in metrics:
            print(f"\n📊 Denormalized Metrics (actual speed values in mph):")
            print(f"  MSE:  {metrics['mse_denorm']:.6f} mph²")
            print(f"  RMSE: {metrics['rmse_denorm']:.4f} mph")
            print(f"  MAE:  {metrics['mae_denorm']:.4f} mph")
            print(f"  MAPE: {metrics['mape']:.2f}%")
            print(f"  R²:   {metrics['r2_denorm']:.6f}")

            print(f"\n📈 True Values Statistics:")
            stats = metrics['y_true_denorm_stats']
            print(f"  Min:  {stats['min']:.2f} mph")
            print(f"  Max:  {stats['max']:.2f} mph")
            print(f"  Mean: {stats['mean']:.2f} mph")
            print(f"  Std:  {stats['std']:.2f} mph")

            print(f"\n🔮 Predicted Values Statistics:")
            stats = metrics['y_pred_denorm_stats']
            print(f"  Min:  {stats['min']:.2f} mph")
            print(f"  Max:  {stats['max']:.2f} mph")
            print(f"  Mean: {stats['mean']:.2f} mph")
            print(f"  Std:  {stats['std']:.2f} mph")


# ============================================================================
# PHASE 4: TEMPORAL VALIDATION
# ============================================================================

class TemporalValidation:
    """Time-series specific tests"""

    @staticmethod
    def test_temporal_consistency(y_pred: np.ndarray,
                                 max_allowed_jump=0.15) -> Dict:
        """Check prediction smoothness over time"""
        results = {}

        # Flatten to time series
        y_pred_flat = y_pred.reshape(-1)

        # Calculate differences between consecutive time steps
        diffs = np.abs(np.diff(y_pred_flat))

        results['max_jump'] = float(diffs.max())
        results['mean_jump'] = float(diffs.mean())
        results['std_jump'] = float(diffs.std())
        results['within_threshold'] = (diffs.max() <= max_allowed_jump)
        results['jump_violations'] = int((diffs > max_allowed_jump).sum())

        return results

    @staticmethod
    def test_boundary_predictions(y_pred: np.ndarray,
                                 expected_min=0, expected_max=1) -> Dict:
        """Ensure predictions stay within valid bounds"""
        results = {}

        results['pred_min'] = float(y_pred.min())
        results['pred_max'] = float(y_pred.max())
        results['within_bounds'] = ((y_pred >= expected_min).all() and
                                   (y_pred <= expected_max).all())
        results['below_min'] = (y_pred < expected_min).sum()
        results['above_max'] = (y_pred > expected_max).sum()

        return results

    @staticmethod
    def run_temporal_tests(y_pred: np.ndarray) -> None:
        """Run all temporal tests"""
        print("\n" + "="*70)
        print("PHASE 4: TEMPORAL VALIDATION")
        print("="*70)

        # Test consistency
        print("\n✓ Testing Temporal Consistency...")
        consistency = TemporalValidation.test_temporal_consistency(y_pred)
        print(f"  Max jump between steps: {consistency['max_jump']:.6f}")
        print(f"  Mean jump: {consistency['mean_jump']:.6f}")
        print(f"  Violations: {consistency['jump_violations']}")
        if consistency['within_threshold']:
            print("  ✅ Predictions temporally smooth")
        else:
            print("  ⚠️  Some large jumps detected")

        # Test boundaries
        print("\n✓ Testing Boundary Predictions...")
        bounds = TemporalValidation.test_boundary_predictions(y_pred)
        print(f"  Min prediction: {bounds['pred_min']:.6f}")
        print(f"  Max prediction: {bounds['pred_max']:.6f}")
        print(f"  Below min: {bounds['below_min']}")
        print(f"  Above max: {bounds['above_max']}")
        if bounds['within_bounds']:
            print("  ✅ All predictions within bounds")
        else:
            print("  ❌ Out-of-bounds predictions detected")

        print("\n" + "="*70)


# ============================================================================
# PHASE 5: EDGE CASE TESTING
# ============================================================================

class EdgeCaseTests:
    """Test model behavior on edge cases"""

    @staticmethod
    def test_extreme_values(model, X_test: np.ndarray,
                           scaler=None) -> Dict:
        """Test on extreme traffic conditions"""
        results = {}

        # Create test samples with extreme values
        X_min = X_test.min(axis=(0, 2), keepdims=True)
        X_max = X_test.max(axis=(0, 2), keepdims=True)

        # Samples with minimum values (congestion)
        X_extreme_min = np.full_like(X_test[:10], X_min)
        pred_extreme_min = model.predict(X_extreme_min, verbose=0)
        results['pred_on_min_input'] = {
            'min': float(pred_extreme_min.min()),
            'max': float(pred_extreme_min.max()),
            'mean': float(pred_extreme_min.mean())
        }

        # Samples with maximum values (free flow)
        X_extreme_max = np.full_like(X_test[:10], X_max)
        pred_extreme_max = model.predict(X_extreme_max, verbose=0)
        results['pred_on_max_input'] = {
            'min': float(pred_extreme_max.min()),
            'max': float(pred_extreme_max.max()),
            'mean': float(pred_extreme_max.mean())
        }

        # Check if predictions make sense
        results['min_input_pred_lower'] = (pred_extreme_min.mean() <
                                          pred_extreme_max.mean())

        return results

    @staticmethod
    def test_corrupted_input(model, X_test: np.ndarray,
                            sensor_id=50) -> Dict:
        """Test with missing sensor data"""
        results = {}

        # Corrupt one sensor with mean value
        X_corrupted = X_test.copy()
        X_corrupted[:, :, sensor_id] = 0.5  # Replace with mean

        pred_clean = model.predict(X_test[:10], verbose=0)
        pred_corrupted = model.predict(X_corrupted[:10], verbose=0)

        # Check difference
        pred_diff = np.abs(pred_corrupted - pred_clean).mean()

        results['sensor_corrupted'] = sensor_id
        results['avg_prediction_change'] = float(pred_diff)
        results['predictions_still_valid'] = (
            (pred_corrupted >= 0).all() and (pred_corrupted <= 1).all()
        )

        return results

    @staticmethod
    def run_edge_case_tests(model, X_test: np.ndarray,
                           scaler=None) -> None:
        """Run all edge case tests"""
        print("\n" + "="*70)
        print("PHASE 5: EDGE CASE TESTING")
        print("="*70)

        print("\n✓ Testing Extreme Values...")
        extreme = EdgeCaseTests.test_extreme_values(model, X_test, scaler)
        print(f"  Congested traffic predictions:")
        print(f"    Mean: {extreme['pred_on_min_input']['mean']:.6f}")
        print(f"  Free flow predictions:")
        print(f"    Mean: {extreme['pred_on_max_input']['mean']:.6f}")
        if extreme['min_input_pred_lower']:
            print("  ✅ Model correctly predicts lower speeds for congestion")

        print("\n✓ Testing Corrupted Input...")
        corrupted = EdgeCaseTests.test_corrupted_input(model, X_test)
        print(f"  Sensor {corrupted['sensor_corrupted']} corruption:")
        print(f"  Avg prediction change: {corrupted['avg_prediction_change']:.6f}")
        if corrupted['predictions_still_valid']:
            print("  ✅ Model handles corrupted input gracefully")

        print("\n" + "="*70)


# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def plot_predictions_vs_actual(y_true: np.ndarray, y_pred: np.ndarray,
                              sample_idx=0, timesteps=20):
    """Visualize predictions"""
    plt.figure(figsize=(14, 5))

    # Plot 1: Time series comparison
    plt.subplot(1, 2, 1)
    true_series = y_true[sample_idx, :timesteps, 0]
    pred_series = y_pred[sample_idx, :timesteps, 0]

    plt.plot(true_series, 'b-o', label='Actual', linewidth=2)
    plt.plot(pred_series, 'r--s', label='Predicted', linewidth=2)
    plt.xlabel('Time Steps')
    plt.ylabel('Normalized Speed')
    plt.title('Predictions vs Actual (Sample Series)')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Plot 2: Scatter plot
    plt.subplot(1, 2, 2)
    plt.scatter(y_true.flatten(), y_pred.flatten(), alpha=0.3, s=1)

    # Perfect prediction line
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', label='Perfect Prediction')

    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title('Prediction Scatter Plot')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()


def run_complete_testing_suite(model, X_train, X_val, X_test,
                              y_train, y_val, y_test,
                              scaler=None, n_sensors=325) -> None:
    """Run all testing phases

    Args:
        model: Trained CNN model
        X_train, X_val, X_test: Input data
        y_train, y_val, y_test: Target data
        scaler: Fitted MinMaxScaler
        n_sensors: Number of sensors/features (default 325 for PEMS Bay)
    """
    print("\n" + "#"*70)
    print("#" + " "*68 + "#")
    print("#" + "  CNN MODEL COMPREHENSIVE TESTING SUITE".center(68) + "#")
    print("#" + " "*68 + "#")
    print("#"*70)

    # Phase 1: Data Quality
    DataQualityTests.run_all_tests(X_train, X_val, X_test,
                                   y_train, y_val, y_test)

    # Phase 3: Performance Metrics
    print("\n" + "="*70)
    print("PHASE 3: PERFORMANCE METRICS EVALUATION")
    print("="*70)

    # Test set predictions
    y_pred_test = model.predict(X_test, verbose=0)
    metrics_test = PerformanceMetrics.calculate_metrics(y_test, y_pred_test, scaler, n_sensors)
    PerformanceMetrics.print_metrics_report(metrics_test, "TEST SET")

    # Validation set predictions
    y_pred_val = model.predict(X_val, verbose=0)
    metrics_val = PerformanceMetrics.calculate_metrics(y_val, y_pred_val, scaler, n_sensors)
    PerformanceMetrics.print_metrics_report(metrics_val, "VALIDATION SET")

    # Phase 4: Temporal Validation
    TemporalValidation.run_temporal_tests(y_pred_test)

    # Phase 5: Edge Cases
    EdgeCaseTests.run_edge_case_tests(model, X_test, scaler)

    # Final Summary
    print("\n" + "#"*70)
    print("#" + " "*68 + "#")
    print("#" + "✅ TESTING SUITE COMPLETED ✅".center(68) + "#")
    print("#" + " "*68 + "#")
    print("#"*70)

    print("\n📊 KEY FINDINGS:")
    print(f"  • Test RMSE: {metrics_test['rmse_denorm']:.4f} mph")
    print(f"  • Test MAE: {metrics_test['mae_denorm']:.4f} mph")
    print(f"  • Test MAPE: {metrics_test['mape']:.2f}%")
    print(f"  • Test R²: {metrics_test['r2_denorm']:.6f}")
    print(f"\n  • Overfitting gap (Val-Train RMSE): TBD (needs train predictions)")
    print(f"  • Model ready for production: Check against success criteria ✓")


In [32]:
# # Import and run tests
# from cnn_model_testing import run_complete_testing_suite

# After training your model:
run_complete_testing_suite(
    model,
    X_train, X_val, X_test,
    y_train, y_val, y_test,
    scaler
)


######################################################################
#                                                                    #
#                CNN MODEL COMPREHENSIVE TESTING SUITE               #
#                                                                    #
######################################################################

PHASE 1: DATA QUALITY TESTING

✓ Testing Data Completeness...
  ✅ No NaN values found

✓ Testing Data Ranges...
  X range: [0.0000, 1.0000]
  y range: [0.0000, 1.0000]
  ✅ Data in expected range

✓ Testing Data Shapes...
  X_train: (36465, 12, 325)
  y_train: (36465, 12, 325)
  X_val:   (5209, 12, 325)
  X_test:  (10419, 12, 325)
  ✅ All shapes correct

✓ Testing Data Distribution...
  X mean: 0.8258 ± 0.1334
  y mean: 0.8258 ± 0.1334
  X outliers: 3.55%
  y outliers: 3.55%
  ✅ Outlier percentage reasonable

✅ ALL DATA QUALITY TESTS PASSED

PHASE 3: PERFORMANCE METRICS EVALUATION

PERFORMANCE METRICS - TEST SET

📊 Normalized Metrics (0

In [19]:
# Save the trained model
model_save_path = "/content/drive/MyDrive/PEMS_BAY/cnn_traffic_model.keras" # Or choose a different path in your Drive

try:
    model.save(model_save_path)
    print(f"✅ Model saved successfully to: {model_save_path}")
except Exception as e:
    print(f"❌ Error saving model: {e}")

✅ Model saved successfully to: /content/drive/MyDrive/PEMS_BAY/cnn_traffic_model.keras
